# This is the first step in the pipeline
### Spots are detected in this notebook. The input file is expected to be in the zarr format 

In [12]:
import pandas as pd
import time
import os
import sys
import zarr
import napari 
import dask.array as da 

pythonPackagePath = os.path.abspath('../src/')
sys.path.append(pythonPackagePath)
from parallel import Detector
from gaussian_visualization import visualize_3D_gaussians

### Do not change the code in cell below 

In [13]:
# This assumes that your notebook is inside 'Jupyter Notebooks', which is at the same level as 'test_data'
base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'movie_data')
# base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'test_movie_1')

zarr_directory = 'zarr_file/all_channels_data'
zarr_full_path = os.path.join(base_dir, zarr_directory)

save_directory = 'datasets'
save_directory_full = os.path.join(base_dir, save_directory)

## Follow the Instructions below to run through the notebook properly 

The purpose of this notebook is to perform spot detection on your full movie. The movie is expected to be a 3 channel movie which is saved as a zarr object. If your movie is not a zarr object you can convert it to a zarr object by running and following the steps provided to you under Final/Data Preparation/full_movie_to_zarr.ipynb

**For Initialising the Detector Object** 

1. Detector object is the main object for which you will setup the parameters to work in this notebook. 
2. **zarr_obj**: is the object which efficiently stores the movie 
3. **save_directory**: is fixed and does not need to be changed. However, for reference the files from this notebook will be outputted and saved in Final/movie_data/datasets directory 
4. **spot_intensity**: This is the minimum intensity which a spot will have in your movie. Anything below this can be called noise/background. You can determine the spot_intensity using fiji or napari to determine minimum bright spots which are of interest. If you set a spot_intensity too high very few spots will be detected, however, if you set a intensity too low a lot of spots including a lot of noise will be detected. 
5. **dist_between_spots**: this distance divided by 2 is the minimum distance that should exist between spots in pixels. For example if you set this to 10 then all spots within 5 pixels of the center of your spot will be dropped(to understand which spot is dropped you can refer to the source code in the Final/src/gaussian_fitting.py file)
6. **sigma_estimations**: This is the spread/radius of your spots from the center. It is entered in pixels and follows the order [z,y,x]. If you expect your spot to have a radius of 4 in z and 2 in x and y then you should enter [4,2,2]. To determine the spread/radius of your spot you can visualise in fiji and look at the metadata to understand the pixel radius. 
7. **n_jobs**: Detector class allows for parallel processing and the number of cores you want to use can be determined here. You can set it to -1 and it will use all_cores - 1 for processing. It is important to allow for parallel processing else for larger movies it will take a lot of time. 
8. **channel_to_detect**: The number of channel to detect. Convention is 1 for channel 1, 2 for channel 2 and 3 for channel 3. The detector object can only detect one channel at a time. 

**For running processing on frames** (run_parallel_frame_processing)

1. **max_frames**: the maximum frames to process. This can be useful when you just want to test your parameters selected for the Detector object like spot_intensity, dist_between_spots and sigma_estimates. 
2. **all_frames**: If all frames is set to True then all frames are processed and the **max_frames** parameter is ignored. It is recommended to initially decide all the parameters on a subset of frames and then move onto this step as it may take a lot of time for larger movies. 


**Note**

-> Cores to be utilized can be increased as available. Keep in mind that limitation can be posed by the RAM of your machine. As more cores are utilized more RAM is needed. 

-> Detection can be only performed on 1 channel at a time


## Set all parameters in the below cell 

In [14]:
#refer to the above cell for explanation of each parameter 
spot_intensity = 180
dist_between_spots = 10
sigma_estimations = [4,2,2]
n_jobs = -1
channel_to_detect = 3 
max_frames = 2 
all_frames = True

In [15]:
#Import the zarr file by adding file path in read mode
z2 = zarr.open(zarr_full_path, mode='r')

In [16]:
z2.info

Type,zarr.core.Array
Data type,uint16
Shape,"(130, 3, 75, 150, 275)"
Chunk shape,"(1, 1, 75, 150, 275)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,2413125000 (2.2G)
No. bytes stored,656696875 (626.3M)
Storage ratio,3.7


In [17]:
frames = z2.shape[0]
print(f'the number of frames are {frames}')

the number of frames are 130


## In the below cell Detector object is initilized to perform detection. More details on the Detector object can be attained by the following line of code: 
**copy and paste in a new cell**

?Detector

In [18]:
detector = Detector(zarr_obj = z2, 
                    save_directory = save_directory_full, 
                    spot_intensity = spot_intensity, 
                    dist_between_spots = dist_between_spots, 
                    sigma_estimations = sigma_estimations, n_jobs = n_jobs, channel_to_detect = channel_to_detect)

In [19]:
#the following function returns the dataframe and also saves it to the provided path in pkl format
#set all_frames = True, to process all the time frames 
#max_frames is useful when you just want to perform detection on a subset of frames. 
#Note: when all_frames= True then max_frames is ignored 
df = detector.run_parallel_frame_processing(max_frames = max_frames, all_frames = all_frames)

Processing frames:   4%|▍         | 5/130 [00:03<01:08,  1.84it/s]

the number of times the gaussian fitting worked was 311 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 298 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 305 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 328 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 297 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 318 and the number of times the gaussian did not fit was 0


Processing frames:   8%|▊         | 11/130 [00:03<00:23,  5.05it/s]

the number of times the gaussian fitting worked was 308 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 302 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 323 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 309 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 299 and the number of times the gaussian did not fit was 0


Processing frames:  11%|█         | 14/130 [00:03<00:16,  7.13it/s]

...

the number of times the gaussian fitting worked was 202 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 208 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 203 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 216 and the number of times the gaussian did not fit was 0


Processing frames:  99%|█████████▉| 129/130 [00:23<00:00,  9.72it/s]

the number of times the gaussian fitting worked was 231 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 214 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 210 and the number of times the gaussian did not fit was 1
the number of times the gaussian fitting worked was 216 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 214 and the number of times the gaussian did not fit was 0


Processing frames: 100%|██████████| 130/130 [00:23<00:00,  5.55it/s]


the number of times the gaussian fitting worked was 224 and the number of times the gaussian did not fit was 0


# Visualising the Output
## Labels are only for time frame 0, for all z slices 

## Below you can see detected spots as masks on the original image and can adjust detection parameters if you think spots are not detected correctly 

### Once you are in the napari viewer you should adjust the contrast and the opacity to make sure both the masks and the raw movie is visible properly.  

In [10]:
# Make a mask of the detections
masks = visualize_3D_gaussians(zarr_obj = z2, gaussians_df = df)

# Create a napari viewer
viewer = napari.Viewer()

#access channel 3 only from zarr array 
dask_array = da.from_zarr(z2)

#the axis arrangement is (t,c,z,y,x)
# importing the channel_to_detect
detection_channel = dask_array[:,:,:,:,:]

# which channel to show
visibility_mask = [False, False, False]
visibility_mask[channel_to_detect-1] = True

# Add the 4D stack to the viewer
# Can change the names of the channels as needed
layer_raw = viewer.add_image(detection_channel, channel_axis = 1, name = ['channel 1', 'channel 2', 'channel 3'], interpolation3d = 'nearest', blending = 'additive', colormap = 'magenta', visible = visibility_mask)
# layer_raw = viewer.add_image(detection_channel, channel_axis = 1, name = ['detection channel'], interpolation3d = 'nearest', blending = 'additive', colormap = 'magenta')

# layer_mask = viewer.add_image(masks, name = 'detections mask')
layer_mask = viewer.add_image(masks, name = 'detections', interpolation3d = 'nearest', blending = 'additive', colormap = 'green')

#other useful parameters 
#color_map = list
#contrast_limits = list of list 

# Add Bounding Box
layer_raw[0].bounding_box.visible = True
layer_raw[1].bounding_box.visible = True
layer_raw[2].bounding_box.visible = True



If the detections don't line up well with the spots in the image:
* make sure you are looking at the first time point
* mouse over the spots in napari to get a sense for the intensity of the spots vs background - use the threshold distinguishing spots from background as spot_intensity 
* vary the dist_between_spots: if the detections are at a higher density than the visible spots, increase the dist_between_spots. And vice versa, if you see spots at a higher density than detections, lower the dist_between_spots.
* If the detections are missing larger or smaller spots you can try increasing or decreasing the sigma_estimations. 

# move to 02.filtering_spots for next steps 